# Go to the bottom of this page to find the pairwise comparison form.

# After you are done filling the form don't forget to scroll down and calculate the consistency of your input. Thank you.

In [523]:
from IPython.display import display, Markdown, clear_output, HTML
import ipywidgets as widgets
import numpy as np
from numpy.linalg import eig
import base64
import pandas as pd  

In [524]:
crtr_lst = ['mushrooms','anchovies','pineapple','bacon']     #list of criteria
crtr_num = len(crtr_lst)             #number of criteria

#initiating comparison list variable
comps = []

#determining comparison combinations by looping through
#the list of criteria and appending to comparisons
p = 0
while p < crtr_num:
    for q in range(p+1,crtr_num):
        comps.append([crtr_lst[p],crtr_lst[q]])
        q += 1
    p += 1
#check
#print(comps)
comps_num = len(comps)

#List of options
Opts = ['1 - Equal Importance',
             '2 - Weak or slight',
             '3 - Moderate importance',
             '4 - Moderate plus',
             '5 - Strong importance',
             '6 - Strong plus',
             '7 - Very strong or demonstrated importance',
             '8 - Very, very strong',
             '9 - Extreme importance']

In [525]:
#creating the form using widgets
#this function returns the values (choice and rate) collected from the form and the form widget itself (it)
def dum():
    lb1 = [widgets.Label(value=str(i+1)+'. Pizza topping:') for i in range(comps_num)]
    
    choice = [widgets.RadioButtons(options=[comps[i][0], comps[i][-1]],
                                   description=30*' ', disabled=False) for i in range(comps_num)]
    
    rate = [widgets.Dropdown(options=Opts, value=Opts[0],
                             disabled=False, layout={'width': 'max-content'}) for i in range(comps_num)]
    
    lb2 = [widgets.Label(value='Preferrence level:') for i in range(comps_num)]
    
    #initializing the list of widgets
    x=[]
    #appending (repeating) to the list of widgets to produce the overall form
    for i in range(comps_num):
        x.append(lb1[i])
        x.append(choice[i])
        x.append(lb2[i])
        x.append(rate[i])
        items = [widgets.VBox(children=[lb1[i], choice[i], lb2[i], rate[i]], height='100%')]
        print("_______________________________________________________________________________________________")
    #print(choice[0].value)
    #assigning the overall form to variable 'it'
    it = widgets.VBox(x)
    return choice, rate, it

In [526]:
#Defining the function to construct the comparison matrix
#this function returns the reciprocal matrix
def construct_matrix():
    #Zero matrix with size equal to number of criteria in both dimensions
    empt_mat = np.zeros((crtr_num,crtr_num),float)
    
    #Assigning the value of number of criteria to s for ease of use
    s = crtr_num
    
    #initializing count
    cnt = 0
    
    #loop to create the reciprocal matrix
    for j in range(0,s):
        empt_mat[j,j] = 1.0
        for k in range(j+1,s):
            
            #if the choice between the two pairs is the first one then pairwise comparison (pw_comp) = rate
            if comps[cnt].index(choice[cnt].value) == 0:
                pw_comp = Opts.index(rate[cnt].value) + 1.0  #plus 1 b/c index starts from 0 (maybe better to use dictionary for Opts)
            
            #if the choice between the two pairs is the second one then pairwise comparison (pw_comp) = 1/rate
            else:
                pw_comp = 1/(Opts.index(rate[cnt].value) + 1.0)
            empt_mat[j,k] = pw_comp         #assign pw_comp as the value for the parts of the matrix above the top left to bottom right diagonal
            empt_mat[k,j] = 1 / pw_comp      #assign 1/pw_comp as the value for the parts of the matrix above the top left to bottom right diagonal
            cnt += 1
    
    recip_mat = empt_mat
    #print(empt_mat)
    
    return recip_mat

In [527]:
#Defining function to calculate weights and consistency
def calculate():
    
    comp_mat = construct_matrix()
    s = crtr_num
    
    #Saaty's random consistency index
    ri = [0, 0, 0.58, 0.9, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49]
    
    e_val, e_vec = eig(comp_mat)    #calculating eigen values and vectors
    
    maxindex = np.argmax(e_val)     #to find the flattened index of the maximum value of the eigen value
    p_vec = e_vec[:,maxindex]    #picking corresponding values (based on maxindex) from the eigen vector and assigning them to p_vec
    w = np.round_(p_vec/sum(p_vec),4)   #normalizing weights
    w = np.real(w)
    lmax = (max(e_val)).real          #Lambda max
    ci = (lmax - s) / (s - 1)         #Consistency index
    cr = round(ci / ri[s-1],4)        #consistency ratio
    
    #print(comp_mat,'\ne_val',e_val,'\ne_vec',e_vec,'\np_vec',p_vec,'\nw',w,'\nlmax',lmax)
    return w, cr


In [528]:
#this function runs when the button is clicked, takes nonetype input (no need to input)
#it produces the display and file output
def on_button_clicked(b):
    
    w, cr = calculate()
    
    #intitialize a dictionary with the consistency key:value pair to store the results
    dt={'consistency':cr}
    
    #append the weights after the first key:value pairs (consistency)
    for i in range(0,len(w)):
        dt[crtr_lst[i]]=[w[i]]
    
    df = pd.DataFrame(data = dt)    #convert the dictionary to a dataframe
    
    #write dataframe to file
    o_name = "output"+Name.value+".csv"
    df.to_csv(o_name)
    
    #read it back
    pd.read_csv(o_name).head()
    
    lnk = "Download 'results"+Name.value+".csv' file"
    f_name = "results"+Name.value+".csv"
    
    #with the 'out' display
    with out:
        #if consistency ratio is <= 0.1 give results, if > 0.1 give warning 
        if cr <= 0.1:
            out.clear_output()          #clear the 'out' display every button press
            print("Consistency: ", cr)
            for i in range(0,len(w)):
                print("\nWeight of ",crtr_lst[i],": ",w[i])
            
            display(HTML(create_download_link(df, lnk, f_name)))    #run function to display link for download
        else:
            out.clear_output()
            print("Consistency Ratio, Cr, greater than 0.1. It has to be lower than 0.1.")
            print("Please check your answers for consistency and calculate again!")
            print("Consistency: ", cr)



In [529]:
#function to create a download link for the result file
def create_download_link( df, title, filename):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return html

In [530]:
#html code to avoid display truncation in notebook

In [531]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:1000000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [532]:
display(Markdown('# Start filling the form here.'))

#calculate button widget
button = widgets.Button(description ='Calculate', disabled = False, button_style = 'warning')

#'out' display to show results in notebook
out = widgets.Output(layout={'border': '1px solid black'})

lb3 = widgets.Label(value='Name of Stakeholder Group')

#Input for name of stakeholder filling the form
Name = widgets.Dropdown(options=['BZK', 'GBB', 'Citizens', 'Housing Association', 'Municipality'],
                        value='BZK', disabled=False)

choice, rate, form = dum()
display(lb3, Name)          #display the label and name input dropdown widgets
display(form)               #display the form widget
display(button, out)        #display the button and the 'out' display not yet populated with results (to be populated after button click)

button.on_click(on_button_clicked)      #run function on_button_clicked with the click of the calculate button

import warnings
warnings.filterwarnings("ignore")      #to avoid error message for complex numbers

# Start filling the form here.

_______________________________________________________________________________________________
_______________________________________________________________________________________________
_______________________________________________________________________________________________
_______________________________________________________________________________________________
_______________________________________________________________________________________________
_______________________________________________________________________________________________


Label(value='Name of Stakeholder Group')

Dropdown(options=('BZK', 'GBB', 'Citizens', 'Housing Association', 'Municipality'), value='BZK')

Button(button_style='warning', description='Calculate', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))

# Don't forget to calculate and check consistency before downloading data.
# Thank you!

In [533]:
#from IPython.display import FileLink
#FileLink('/home/jovyan/output.txt')

In [534]:
#cd ~ https://github.com/Bisrat-Araya/world-of-wonders